In [46]:
def mapl(f, xs): return list(map(f, xs))

In [159]:
with open("./inputs/day10.txt") as f:
    inp = f.readlines()

def parse_line(line):
    target, *buttons, joltages = line.strip().split()
    target = sum(map(lambda t: 2**t[0] if t[1] == '#' else 0, enumerate(target[1:-1])))
    buttons = [b.replace(")", ",)") for b in buttons]
    buttons = list(map(eval, buttons))
    buttons_binary = [sum(map(lambda x: 2**x, b)) for b in buttons]
    joltages = eval(joltages.replace('{', '(').replace('}', ')'))
    return target, buttons_binary, buttons, joltages

In [160]:
def validate_target(current, subsets):
    if not subsets:
        return 100 if current else 0
    
    head, tail = subsets[0], subsets[1:]
    with_first = 1 + validate_target(current ^ head, tail)
    without_first = validate_target(current, tail)
    return min(with_first, without_first)

In [161]:
part1 = 0
for target, buttons, _, _ in map(parse_line, inp):
    part1 += validate_target(target, buttons)
print(part1)

578


In [163]:
part2_inputs = [(cs, b) for _, _, cs, b in mapl(parse_line, inp)]

In [ ]:
from ortools.sat.python import cp_model
import numpy as np

def solve_using_csp(switches, target):

    A = np.zeros((len(switches), len(target)))
    for i, col in enumerate(switches):
        for j in col:
            A[i, j] = 1

    constraints = {}
    for i in range(len(target)):
        constraints[tuple(A[:, i].nonzero()[0].tolist())] = target[i]


    max_values = {}
    for vars, mx in constraints.items():
        for v in vars:
            max_values[v] = min(max_values.get(v, mx), mx)

    model = cp_model.CpModel()
    vars = [model.new_int_var(0, max_values[i], f"a_{i}") for i in range(len(switches))]

    for cs, b in constraints.items():
        model.add(sum(vars[i] for i in cs) == b)
    model.minimize(sum(vars))
    solver = cp_model.CpSolver()
    status = solver.solve(model)

    if status != cp_model.OPTIMAL:
        raise ValueError("Didn't find the optimal")
    return sum(map(solver.value, vars))

In [164]:
sum(solve_using_csp(*inp) for inp in part2_inputs)

20709